In [77]:
### Google Secret Manager Client Instalation


# !pip install google-cloud-secret-manager
# Install gcloud: https://cloud.google.com/sdk/docs/install#windows
# Setup the credential: gcloud auth application-default login

In [178]:
import json
from google.cloud import secretmanager
import redis

import numpy as np

from tqdm.notebook import tqdm

In [74]:
def get_secret_value(project_id = "652914548272", secret_id = "redis", key = "redis-test"):
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()
    
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    payload = response.payload.data.decode("UTF-8")
    configs = eval(payload)

    config_key = configs[key]

    return config_key


In [75]:
config_redis = get_secret_value(key = "redis-test")

In [76]:
r = redis.Redis(
  host=config_redis["host"],
  port=config_redis["port"],
  password=config_redis["password"])

# Load Index Json

In [240]:
with open("index_sample.json", 'r+') as f:
    data_json = json.load(f)

In [241]:
data_json.keys()

dict_keys(['document_size', 'doc_ids_list', 'ft', '14', '92', 'havel', 'famili', 'arian', 'genillard', 'anthoni', 'robinson', 'set', 'scene', 'czechoslovak', 'elect', 'futur', 'czechoslovakia', 'econom', 'reform', 'fate', 'czech', 'slovak', 'feder', 'state', 'main', 'issu', 'debat', 'hard', 'fought', 'gener', 'campaign', 'involv', '40', 'polit', 'parti', 'began', 'yesterday', 'open', 'shot', 'fire', 'mr', 'vaclav', 'presid', 'republ', 'address', 'nation', 'bratislava', 'capit', 'warn', 'unconsid', 'danger', 'attempt', 'break', 'countri', 'fragil', 'move', 'union', 'reconstitut', 'vagu', 'dualist', 'lead', 'chaotic', 'exact', 'high', 'price', 'similar', 'up', 'yugoslavia', 'soviet', 'presidenti', 'direct', 'nationalist', 'strive', 'greater', 'autonomi', 'hold', '16m', 'popul', 'vladimir', 'meciar', 'movement', 'democrat', 'slovakia', 'expect', 'emerg', 'largest', 'june', '5', 'promis', 'adopt', 'constitut', 'declar', 'sovereignti', 'win', 'poll', 'place', 'politician', '34', 'cent', 'su

# Create a new format index

In [168]:
data = {
    "document_size": data_json['document_size'],
    "doc_ids_list": data_json['doc_ids_list']
}

data['terms'] = {}

In [169]:
for keyword in data_json.keys():
    if keyword not in ['document_size', 'doc_ids_list']:
        data['terms'][keyword] = data_json[keyword]

# Push Index to Redis

In [208]:
r.set("document_size", str(data_json['document_size']))

True

In [209]:
r.set("doc_ids_list", str(data_json['doc_ids_list']))

True

In [181]:
# Store term positions in documents
for term in tqdm(data_json.keys()):
    if term not in ['document_size', 'doc_ids_list']:
        r.set(term, str(data_json['term'])) #1263/39310

# Retrieve value from Redis

In [219]:
def get_val(r, key):
    value = r.get(key)
    value = value.decode()
    value = eval(value)
    return value

In [223]:
get_val(r, 'document_size')

{'0': 5000}

In [224]:
term = 'ft'

val = get_val(r, term)